# Madrigals

First we downloaded the dataset from the British Library about the History of Printed Music. 

We then load the dataset using the pandas library in order to examine our data

In [2]:
import pandas as pd

In [4]:
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''

In [5]:
df = pd.read_csv("C:/Users/const/Downloads/MusicResearcherFormat_201505_csv/detailedrecords.csv",converters={'first_column': convert_dtype,'second_column': convert_dtype})

C:\Users\const\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (7,14,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


From the big dataset, we extract only rows where the relevant columns contain the characters "madrigal"

In [8]:
df_madri = df.loc[df["Subject/genre terms"].str.contains('madrigal', case=False, na=False) | df["Title"].str.contains('madrigal', case=False, na=False) | df["Other titles"].str.contains('madrigal', case=False, na=False) | df["Notes"].str.contains('madrigal', case=False, na=False)]

In [9]:
df_madri

,BL record ID,Composer,Composer life dates,Title,Standardised title,Other titles,Other names,Publication date (standardised),Publication date (not standardised),Country of publication,...,Contents,Referenced in,Subject/genre terms,Physical description,Series title,Number within series,ISBN,ISMN,Publisher number,BL shelfmark
3722,4166081,"Adlam, Frank",NaN,Winter stern hath loosed his Grip. <Madrigal f...,NaN,NaN,NaN,1913,1913,England,...,NaN,NaN,NaN,"3 pages, 8°",Choruses for equal Voices,no. 1387 [Choruses for equal Voices],NaN,NaN,NaN,mDON1781 ; E.861./1387
3804,4166165,"Adler, Samuel",1928-,Three Madrigals. For four-part chorus of mixed...,NaN,NaN,NaN,1958,1958,United States,...,NaN,NaN,NaN,"11 pages, 8°",NaN,NaN,NaN,NaN,NaN,F.1744.v.(1.)
3932,4166293,"Adriani, Francesco",NaN,Il Primo Libro de Madrigali a Cinque Voci ... ...,NaN,NaN,NaN,1570,1570,Italy,...,NaN,NaN,NaN,4°,NaN,NaN,NaN,NaN,NaN,D.148
3933,4166294,"Adriani, Francesco",NaN,Il Secondo Libro de Madrigali a Cinque Voci .....,NaN,NaN,NaN,1570,1570,Italy,...,NaN,NaN,NaN,4°,NaN,NaN,NaN,NaN,NaN,D.148.a
3934,4166295,"Adriansen, Emanuel",approximately 1554-1604,"Luitmuziek ... Een keuze van fantasieën, danse...",NaN,NaN,"Spiessons, Godelieve",1966,1966,Belgium,...,NaN,NaN,NaN,"xvi, 97 pages, facsimiles, folio",Monumenta musicæ belgicæ,no. 10 [Monumenta musicæ belgicæ],NaN,NaN,NaN,H.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044411,16837423,"Bennet, John",active 1599-1614,All creatures now are merry-minded : (SSATB) ;...,NaN,Triumphes of Oriana,"Wilkes, Roger, (Musician) [editor]",2014,©2014,England,...,NaN,NaN,"Choruses, Secular (Mixed voices)--Scores ; Mad...","1 score (12 pages) + 9 parts, 30 cm",Editions of renaissance music,NaN,NaN,9790570570317 ; 9790570570324,TR004,G.1535.l.(1.)
1044413,16837433,"Jones, Robert",active 1597-1615,"Fair Oriana, seeming to wink : (SSAATB) : from...",NaN,Triumphes of Oriana,"Wilkes, Roger, (Musician) [editor]",2014,©2014,England,...,NaN,NaN,"Choruses, Secular (Mixed voices)--Scores ; Mad...","1 score (12 pages) + 14 parts, 30 cm",Editions of renaissance music,NaN,NaN,9790570570300,TR021,G.1535.l.(4.)
1044421,16840548,"Jackson, William",1730-1803,"Ten duets : for sopranos, tenors, or soprano &...",Vocal music. Selections,Duets ; Canzonets ; Pastorals ; Madrigals,"Jackson, William, op. 9 ; Jackson, William, op...",2014,©2013,England,...,"From Twelve canzonets, op. 9. Time has not thi...",NaN,Vocal duets with continuo,"2 scores (44, 39 pages), 30 cm",NaN,NaN,NaN,9790708105701,Jac 1,G.804.ll.(11.)
1044497,16874797,"Tomkins, Thomas",1572-1656,Dear Lord of life,NaN,NaN,"Burke, James [editor]",2014,©2014,England,...,NaN,NaN,"Anthems ; Choruses, Sacred (Mixed voices, 6 pa...","1 score (15 pages), 30 cm",Church Music Society reprints,no. 132 [Church Music Society reprints],9780193954014,NaN,NaN,E.1617./132


To organize our data and extract only the works that have been composed in our period of interest (1530 - 1650), we decided to clean the "Composer life dates" column, adding manually those that are missing. 

We first extract only the column "Composer" and "Composer life dates"


Philippe Verdelot 1480
-
Scarlatti 1660-1725

To be a bit large we select composers born after 1450 and dead before 1730

In [11]:
comp=df_madri[["Composer", "Composer life dates"]]
comp =comp.sort_values(["Composer"])
comp = comp.drop_duplicates()
comp

,Composer,Composer life dates
879004,16th century madrigals,NaN
298323,A. L. (Amelia Lehmann),NaN
3722,"Adlam, Frank",NaN
3804,"Adler, Samuel",1928-
3935,"Adriaensen, Emanuel",approximately 1550-1604
...,...,...
1025293,"Zarlino, Gioseffo",1517-1590
598665,"Zoilo, Annibale",NaN
598666,"Zoilo, Cesare",NaN
598868,"Zucca, Mana, pseud",NaN


In [7]:
comp.to_csv("composers_dates.csv")

After exporting our csv and correcting it manually, we import it back into our project. 

In [12]:
composers = pd.read_csv("composers_dates_cleaned.csv")

In [13]:
composers = composers.dropna()
composers.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
composers

,Composer,Composer life dates
1,A. L. (Amelia Lehmann),1838-1903
2,"Adler, Samuel",1928-
3,"Adriaensen, Emanuel",1550-1604
4,"Adriani, Francesco",1539-1575
5,"Adriansen, Emanuel",1554-1604
...,...,...
937,"Zanotti, Camillo",1545-1591
938,"Zarlino, Gioseffo",1517-1590
939,"Zoilo, Annibale",1537-1592
940,"Zoilo, Cesare",1584-1622


In [15]:
dict_composers = composers.set_index("Composer").to_dict()
#dict_composers

Then we select only composers born in the right period: after the first composer of madrigals Philippe Verdelot in 1480, and dead before the last composer of madrigals of our period of interest: Scarlatti in 1725. To include cases of composers born at the extremities of this time frame, we delimited the time period between 1450 and 1730.

In [16]:
comp_madrigals = {}
for composer in dict_composers['Composer life dates']:
    dates = dict_composers['Composer life dates'][composer]
    dates = dates.strip("- ")
    dates_list = dates.split("-")
    if len(dates_list)>1:
        if int(dates_list[0])>=1450 and int(dates_list[1])<=1730:
            comp_madrigals[composer]=dates
    else:
        if int(dates_list[0])>=1450 and int(dates_list[0])<=1730:
            comp_madrigals[composer]=dates
   
print(len(comp_madrigals))
print(comp_madrigals)

369
{'Adriaensen, Emanuel': '1550-1604', 'Adriani, Francesco': '1539-1575', 'Adriansen, Emanuel': '1554-1604', 'Agazzari, Agostino': '1578-1640', 'Agostini, Lodovico': '1534-1590', 'Aichinger, Gregor': '1564-1628', 'Aleotti, Vittoria': '1573-1620', 'Alison, Richard': '1565-1610', 'Anerio, Felice': '1560-1614', 'Anerio, Giovanni Francesco': '1567-1630', 'Animuccia, Giovanni': '1500-1571', 'Annibale, Padovano': '1527-1575', 'Arcadelt, Jacob': '1505-1568', 'Archilei, Antonio': '1500-1612', 'Asola, Giovanni Matteo': '1532-1609', 'Azzaiolo, Filippo': '1530-1569', 'Baccusi, Ippolito': '1550-1609', 'Balsamino, Simone': '1596', 'Banchieri, Adriano': '1568-1634', 'Barbarino, Bartolomeo': '1617', 'Barberiis, Melchiorre de': '1500-1549', "Bardi, Giovanni de'": '1534-1612', 'Bassani, Giovanni Battista': '1650-1716', 'Bassano, Giovanni': '1558-1617', 'Bateson, Thomas': '1630', 'Bati, Luca': '1546-1608', 'Beauvarlet, Henricus': '1575-1623', 'Becchi, Antonio di': '1522-1568', 'Belli, Girolamo': '1552

# Working on composers of interest: 
We want to merge our dictionary with information extracted from Wikidata.
First we need to change the name in format "surname, name" to "name surname" which is the one adopted for labels on Wikidata.

In [17]:
dict_comp_madrigals = {}
for el in comp_madrigals.keys():
    
    #exchanging "surname, name" for "name surname"
    if len(el.split(", "))>1:
        splitted = el.split(", ")
        name = splitted[1].strip("'")
        surname = splitted[0].strip("'")
        dict_comp_madrigals[name+" "+surname] = comp_madrigals[el]
        
    #some entities, like "Henry VIII", are not in "surname, name" format, so they should be added as is.
    else:
        dict_comp_madrigals[el] = comp_madrigals[el]
        
#printing the dictionary        
dict_comp_madrigals


{'Emanuel Adriaensen': '1550-1604',
 'Francesco Adriani': '1539-1575',
 'Emanuel Adriansen': '1554-1604',
 'Agostino Agazzari': '1578-1640',
 'Lodovico Agostini': '1534-1590',
 'Gregor Aichinger': '1564-1628',
 'Vittoria Aleotti': '1573-1620',
 'Richard Alison': '1565-1610',
 'Felice Anerio': '1560-1614',
 'Giovanni Francesco Anerio': '1567-1630',
 'Giovanni Animuccia': '1500-1571',
 'Padovano Annibale': '1527-1575',
 'Jacob Arcadelt': '1505-1568',
 'Antonio Archilei': '1500-1612',
 'Giovanni Matteo Asola': '1532-1609',
 'Filippo Azzaiolo': '1530-1569',
 'Ippolito Baccusi': '1550-1609',
 'Simone Balsamino': '1596',
 'Adriano Banchieri': '1568-1634',
 'Bartolomeo Barbarino': '1617',
 'Melchiorre de Barberiis': '1500-1549',
 'Giovanni de Bardi': '1534-1612',
 'Giovanni Battista Bassani': '1650-1716',
 'Giovanni Bassano': '1558-1617',
 'Thomas Bateson': '1630',
 'Luca Bati': '1546-1608',
 'Henricus Beauvarlet': '1575-1623',
 'Antonio di Becchi': '1522-1568',
 'Girolamo Belli': '1552-1620'

### Querying the Wikidata SPARQL endpoint API
When sending a query to the wikidata sparql endpoint api, the full list was too big to handle, so we split it in groups of 100 composers.

We use regular expressions to search for persons which have as occupation "composer" and whose label matches one in the list. 
By groups of 100, the names are concatenated in a big string with the operator OR ("|") and we make sure that the name starts ("^") and ends ("$") exactly with those characters.

Without this condition, the first query returned other composers which were almost homonyms from the ones in our list, for example "Friedrich Nicolaus Bruhns" instead of only "Nicolaus Bruhns".

In [18]:
#one query with all the composers is too long, so we split the list in sub_lists of 100 composers.
list_keys = list(dict_comp_madrigals.keys())
chunked_list_to_be_queried = [list_keys[i:i+100] for i in range(0, len(dict_comp_madrigals), 100)]

In [19]:
#instantiating the final dictionary
dict_composers_wd = {}

In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"

#sending a query for groups of 100 composers
for hundred_composers in chunked_list_to_be_queried:
    to_str = "$|^".join(hundred_composers)

    query_composers = """
    SELECT DISTINCT *
    WHERE {
            ?composer wdt:P106 wd:Q36834;
            rdfs:label ?label.
            FILTER regex(?label, \"^"""+to_str+"""$\" )
            FILTER (langMatches(lang(?label), "EN"))
            }
    """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_composers)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    # manipulate the result
    for result in results["results"]["bindings"]:
        #the dictionary contains as key the name of the composers, 
        #and as value another dictionary containing the future columns of the dataframe "wikidata" and "dates"
        dict_composers_wd[result["label"]["value"]] = {"wikidata":result["composer"]["value"], 
                                                       "dates":dict_comp_madrigals[result["label"]["value"]]}

In [30]:
print(len(dict_composers_wd))
print(dict_composers_wd)

244
{'Nicolaus Bruhns': {'wikidata': 'http://www.wikidata.org/entity/Q57369', 'dates': '1665-1697'}, 'Marc-Antoine Charpentier': {'wikidata': 'http://www.wikidata.org/entity/Q55524', 'dates': '1643-1704'}, 'Girolamo Dalla Casa': {'wikidata': 'http://www.wikidata.org/entity/Q354023', 'dates': '1543-1601'}, 'Arcangelo Corelli': {'wikidata': 'http://www.wikidata.org/entity/Q164475', 'dates': '1653-1713'}, 'John Bennet': {'wikidata': 'http://www.wikidata.org/entity/Q374718', 'dates': '1575-1614'}, 'Maddalena Casulana': {'wikidata': 'http://www.wikidata.org/entity/Q269690', 'dates': '1544-1590'}, 'Agostino Agazzari': {'wikidata': 'http://www.wikidata.org/entity/Q395563', 'dates': '1578-1640'}, 'Adriano Banchieri': {'wikidata': 'http://www.wikidata.org/entity/Q347804', 'dates': '1568-1634'}, 'Scipione Dentice': {'wikidata': 'http://www.wikidata.org/entity/Q290052', 'dates': '1560-1635'}, 'John Dowland': {'wikidata': 'http://www.wikidata.org/entity/Q207355', 'dates': '1563-1626'}, 'William By

We get a dataset of 244 composers out of the 369 in the British Library catalogue, so approximately 2/3 of them.

We now send another query to wikidata to see if we can find other composers of madrigals to add to our dataset.

In [41]:
# prepare the query : 10 random triples
query_new_composers = """
SELECT ?composition ?compositionLabel ?composer ?composerLabel ?birthdate ?deathdate WHERE {
        ?composition wdt:P86 ?composer;
                     wdt:P7937 wd:Q193217.
        ?composer wdt:P569 ?birthdate;
                  wdt:P570 ?deathdate.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }  # labels in English
}

"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(query_new_composers)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

# manipulate the result
for result in results["results"]["bindings"]:
    #Adding the composer only if it doesn't already exist in the dataset
    if result["composerLabel"]["value"] not in dict_composers_wd:
        birthdate = result["birthdate"]["value"].split("-")
        deathdate = result["deathdate"]["value"].split("-")
        print(birthdate[0]+"-"+deathdate[0])
        dict_composers_wd[result["composerLabel"]["value"]] = {"wikidata":result["composer"]["value"],
                                                              "dates":birthdate[0]+"-"+deathdate[0]}

1500-1570


In [42]:
print(len(dict_composers_wd))

245


In [40]:
#TO DELETE IN FINAL VERSION
#temp variable for storing the dict in case wikidata endpoint query doesn't work anymore. 
dict_composers_wd = {'Nicolaus Bruhns': {'wikidata': 'http://www.wikidata.org/entity/Q57369', 'dates': '1665-1697'}, 'Marc-Antoine Charpentier': {'wikidata': 'http://www.wikidata.org/entity/Q55524', 'dates': '1643-1704'}, 'Girolamo Dalla Casa': {'wikidata': 'http://www.wikidata.org/entity/Q354023', 'dates': '1543-1601'}, 'Arcangelo Corelli': {'wikidata': 'http://www.wikidata.org/entity/Q164475', 'dates': '1653-1713'}, 'John Bennet': {'wikidata': 'http://www.wikidata.org/entity/Q374718', 'dates': '1575-1614'}, 'Maddalena Casulana': {'wikidata': 'http://www.wikidata.org/entity/Q269690', 'dates': '1544-1590'}, 'Agostino Agazzari': {'wikidata': 'http://www.wikidata.org/entity/Q395563', 'dates': '1578-1640'}, 'Adriano Banchieri': {'wikidata': 'http://www.wikidata.org/entity/Q347804', 'dates': '1568-1634'}, 'Scipione Dentice': {'wikidata': 'http://www.wikidata.org/entity/Q290052', 'dates': '1560-1635'}, 'John Dowland': {'wikidata': 'http://www.wikidata.org/entity/Q207355', 'dates': '1563-1626'}, 'William Byrd': {'wikidata': 'http://www.wikidata.org/entity/Q208375', 'dates': '1543-1623'}, 'Thomas Campion': {'wikidata': 'http://www.wikidata.org/entity/Q455618', 'dates': '1567-1620'}, "Sigismondo D'India": {'wikidata': 'http://www.wikidata.org/entity/Q457145', 'dates': '1582-1629'}, 'Giulio Caccini': {'wikidata': 'http://www.wikidata.org/entity/Q215308', 'dates': '1551-1618'}, 'Giovanni Battista Bassani': {'wikidata': 'http://www.wikidata.org/entity/Q465565', 'dates': '1650-1716'}, 'Antonio Cifra': {'wikidata': 'http://www.wikidata.org/entity/Q601590', 'dates': '1584-1629'}, 'Baldassare Donato': {'wikidata': 'http://www.wikidata.org/entity/Q804795', 'dates': '1530-1603'}, 'Melchior Borchgrevinck': {'wikidata': 'http://www.wikidata.org/entity/Q469278', 'dates': '1570-1632'}, 'Gregor Aichinger': {'wikidata': 'http://www.wikidata.org/entity/Q633088', 'dates': '1564-1628'}, 'Fabrizio Dentice': {'wikidata': 'http://www.wikidata.org/entity/Q635911', 'dates': '1593-1581'}, 'Hans Brachrogge': {'wikidata': 'http://www.wikidata.org/entity/Q472819', 'dates': '1590-1638'}, 'Giovanni Francesco Anerio': {'wikidata': 'http://www.wikidata.org/entity/Q711181', 'dates': '1567-1630'}, 'Giovanni Bassano': {'wikidata': 'http://www.wikidata.org/entity/Q494292', 'dates': '1558-1617'}, 'Filippo Azzaiolo': {'wikidata': 'http://www.wikidata.org/entity/Q717917', 'dates': '1530-1569'}, 'Felice Anerio': {'wikidata': 'http://www.wikidata.org/entity/Q549044', 'dates': '1560-1614'}, 'Christoph Demantius': {'wikidata': 'http://www.wikidata.org/entity/Q497554', 'dates': '1567-1643'}, 'Giovanni Animuccia': {'wikidata': 'http://www.wikidata.org/entity/Q500507', 'dates': '1500-1571'}, 'Giovanni Croce': {'wikidata': 'http://www.wikidata.org/entity/Q920564', 'dates': '1557-1609'}, 'Lodovico Agostini': {'wikidata': 'http://www.wikidata.org/entity/Q946839', 'dates': '1534-1590'}, 'Tomaso Cecchino': {'wikidata': 'http://www.wikidata.org/entity/Q1117014', 'dates': '1580-1644'}, 'Guillaume Bouzignac': {'wikidata': 'http://www.wikidata.org/entity/Q983567', 'dates': '1587-1643'}, 'Ippolito Baccusi': {'wikidata': 'http://www.wikidata.org/entity/Q1493373', 'dates': '1550-1609'}, 'Francesco Corteccia': {'wikidata': 'http://www.wikidata.org/entity/Q1230392', 'dates': '1502-1571'}, 'Michael East': {'wikidata': 'http://www.wikidata.org/entity/Q1927338', 'dates': '1580-1648'}, 'Richard Dering': {'wikidata': 'http://www.wikidata.org/entity/Q2149323', 'dates': '1580-1630'}, 'Antonio Archilei': {'wikidata': 'http://www.wikidata.org/entity/Q2857177', 'dates': '1500-1612'}, 'Vittoria Aleotti': {'wikidata': 'http://www.wikidata.org/entity/Q2707991', 'dates': '1573-1620'}, 'Nicolao Dorati': {'wikidata': 'http://www.wikidata.org/entity/Q2754386', 'dates': '1513-1593'}, 'Luca Bati': {'wikidata': 'http://www.wikidata.org/entity/Q2424465', 'dates': '1546-1608'}, 'Severo Bonini': {'wikidata': 'http://www.wikidata.org/entity/Q3480834', 'dates': '1582-1663'}, 'Jean de Castro': {'wikidata': 'http://www.wikidata.org/entity/Q3175340', 'dates': '1540-1600'}, 'Domenico Bianchini': {'wikidata': 'http://www.wikidata.org/entity/Q3712917', 'dates': '1510-1576'}, 'Thomas Bateson': {'wikidata': 'http://www.wikidata.org/entity/Q3524896', 'dates': '1630'}, 'Fabio Costantini': {'wikidata': 'http://www.wikidata.org/entity/Q3737659', 'dates': '1570-1644'}, 'Francesco Bianciardi': {'wikidata': 'http://www.wikidata.org/entity/Q3749629', 'dates': '1570-1607'}, 'Giulio Belli': {'wikidata': 'http://www.wikidata.org/entity/Q3769496', 'dates': '1560-1621'}, 'Richard Carlton': {'wikidata': 'http://www.wikidata.org/entity/Q3934778', 'dates': '1558-1638'}, 'Camillo Cortellini': {'wikidata': 'http://www.wikidata.org/entity/Q3651486', 'dates': '1561-1630'}, 'Michael Cavendish': {'wikidata': 'http://www.wikidata.org/entity/Q3856134', 'dates': '1565-1628'}, 'Bartolomeo Barbarino': {'wikidata': 'http://www.wikidata.org/entity/Q4865504', 'dates': '1617'}, 'Joan Brudieu': {'wikidata': 'http://www.wikidata.org/entity/Q4894403', 'dates': '1520-1591'}, 'Girolamo Conversi': {'wikidata': 'http://www.wikidata.org/entity/Q5564760', 'dates': '1572-1575'}, 'Girolamo Belli': {'wikidata': 'http://www.wikidata.org/entity/Q5564752', 'dates': '1552-1620'}, 'Giovanni Battista Conforti': {'wikidata': 'http://www.wikidata.org/entity/Q5563724', 'dates': '1550-1570'}, 'Ippolito Ciera': {'wikidata': 'http://www.wikidata.org/entity/Q6065392', 'dates': '1546-1561'}, 'Thomas Brewer': {'wikidata': 'http://www.wikidata.org/entity/Q7787872', 'dates': '1611-1665'}, 'Simon Boyleau': {'wikidata': 'http://www.wikidata.org/entity/Q7518324', 'dates': '1544-1586'}, 'Simone Balsamino': {'wikidata': 'http://www.wikidata.org/entity/Q7520256', 'dates': '1596'}, 'Giovanni Battista Crivelli': {'wikidata': 'http://www.wikidata.org/entity/Q16206148', 'dates': '1652'}, 'Valerio Bona': {'wikidata': 'http://www.wikidata.org/entity/Q16859665', 'dates': '1560-1620'}, 'Giovanni Contino': {'wikidata': 'http://www.wikidata.org/entity/Q20100441', 'dates': '1513-1574'}, 'Francesco Adriani': {'wikidata': 'http://www.wikidata.org/entity/Q28548877', 'dates': '1539-1575'}, 'William Cobbold': {'wikidata': 'http://www.wikidata.org/entity/Q43234079', 'dates': '1560-1639'}, 'Giovanni Boschetto Boschetti': {'wikidata': 'http://www.wikidata.org/entity/Q55225893', 'dates': '1570-1622'}, 'Lelio Bertani': {'wikidata': 'http://www.wikidata.org/entity/Q112536290', 'dates': '1550-1620'}, 'Giovanni Battista Chinelli': {'wikidata': 'http://www.wikidata.org/entity/Q57271845', 'dates': '1610-1677'}, 'Giovanni Del Turco': {'wikidata': 'http://www.wikidata.org/entity/Q56006649', 'dates': '1577-1647'}, 'Jacopo Corfini': {'wikidata': 'http://www.wikidata.org/entity/Q82008847', 'dates': '1540-1591'}, 'Giovanni Battista Caletti': {'wikidata': 'http://www.wikidata.org/entity/Q96070821', 'dates': '1577-1642'}, 'Giulio Bonagiunta': {'wikidata': 'http://www.wikidata.org/entity/Q108937597', 'dates': '1530-1571'}, 'Giovanni Gabrieli': {'wikidata': 'http://www.wikidata.org/entity/Q34624', 'dates': '1554-1612'}, 'Henry VIII': {'wikidata': 'http://www.wikidata.org/entity/Q38370', 'dates': '1491-1547'}, 'Sebastian Knüpfer': {'wikidata': 'http://www.wikidata.org/entity/Q71496', 'dates': '1633-1676'}, 'Thomas Ford': {'wikidata': 'http://www.wikidata.org/entity/Q354266', 'dates': '1580-1648'}, 'Vincenzo Galilei': {'wikidata': 'http://www.wikidata.org/entity/Q313765', 'dates': '1520-1591'}, 'Josquin des Prez': {'wikidata': 'http://www.wikidata.org/entity/Q143100', 'dates': '1450-1521'}, 'Luca Marenzio': {'wikidata': 'http://www.wikidata.org/entity/Q316532', 'dates': '1553-1599'}, 'Heinrich Isaac': {'wikidata': 'http://www.wikidata.org/entity/Q316668', 'dates': '1450-1517'}, 'Valentin Haussmann': {'wikidata': 'http://www.wikidata.org/entity/Q249689', 'dates': '1560-1614'}, 'Daniel Friderici': {'wikidata': 'http://www.wikidata.org/entity/Q322254', 'dates': '1584-1638'}, 'Biagio Marini': {'wikidata': 'http://www.wikidata.org/entity/Q110224592', 'dates': '1587-1663'}, 'Hans Leo Hassler': {'wikidata': 'http://www.wikidata.org/entity/Q164882', 'dates': '1564-1612'}, 'Johannes Jeep': {'wikidata': 'http://www.wikidata.org/entity/Q428075', 'dates': '1581-1644'}, 'Carlo Gesualdo': {'wikidata': 'http://www.wikidata.org/entity/Q192958', 'dates': '1560-1613'}, 'Stefano Felis': {'wikidata': 'http://www.wikidata.org/entity/Q431549', 'dates': '1550-1603'}, 'Luzzasco Luzzaschi': {'wikidata': 'http://www.wikidata.org/entity/Q445581', 'dates': '1544-1607'}, 'Thomas Lupo': {'wikidata': 'http://www.wikidata.org/entity/Q353855', 'dates': '1571-1628'}, 'John Farmer': {'wikidata': 'http://www.wikidata.org/entity/Q1335446', 'dates': '1570-1591'}, 'Girolamo Frescobaldi': {'wikidata': 'http://www.wikidata.org/entity/Q209212', 'dates': '1583-1643'}, 'Andrea Gabrieli': {'wikidata': 'http://www.wikidata.org/entity/Q217317', 'dates': '1533-1585'}, 'Gioseffo Guami': {'wikidata': 'http://www.wikidata.org/entity/Q610708', 'dates': '1540-1611'}, 'Francisco Guerrero': {'wikidata': 'http://www.wikidata.org/entity/Q470568', 'dates': '1528-1599'}, 'Marco da Gagliano': {'wikidata': 'http://www.wikidata.org/entity/Q540638', 'dates': '1582-1643'}, 'Andreas Hammerschmidt': {'wikidata': 'http://www.wikidata.org/entity/Q497864', 'dates': '1612-1675'}, 'Leonhard Lechner': {'wikidata': 'http://www.wikidata.org/entity/Q683132', 'dates': '1553-1606'}, 'Costanzo Festa': {'wikidata': 'http://www.wikidata.org/entity/Q937584', 'dates': '1490-1545'}, 'Alessandro Grandi': {'wikidata': 'http://www.wikidata.org/entity/Q948575', 'dates': '1590-1630'}, 'Giovanni Giacomo Gastoldi': {'wikidata': 'http://www.wikidata.org/entity/Q964231', 'dates': '1582-1609'}, 'John Hilton the younger': {'wikidata': 'http://www.wikidata.org/entity/Q1289313', 'dates': '1599-1657'}, 'Ruggiero Giovannelli': {'wikidata': 'http://www.wikidata.org/entity/Q1369480', 'dates': '1560-1625'}, 'Giles Farnaby': {'wikidata': 'http://www.wikidata.org/entity/Q1343764', 'dates': '1565-1640'}, 'Giovanni Piero Manenti': {'wikidata': 'http://www.wikidata.org/entity/Q1526316', 'dates': '1535-1597'}, 'Anthony Holborne': {'wikidata': 'http://www.wikidata.org/entity/Q1676220', 'dates': '1545-1602'}, 'Giovanni Pietro Flaccomio': {'wikidata': 'http://www.wikidata.org/entity/Q1526322', 'dates': '1565-1617'}, 'Giovanni de Macque': {'wikidata': 'http://www.wikidata.org/entity/Q1403614', 'dates': '1550-1614'}, 'Simon Gintzler': {'wikidata': 'http://www.wikidata.org/entity/Q1695832', 'dates': '1490-1547'}, 'Noé Faignient': {'wikidata': 'http://www.wikidata.org/entity/Q2134295', 'dates': '1537-1578'}, 'George Kirbye': {'wikidata': 'http://www.wikidata.org/entity/Q1997700', 'dates': '1565-1634'}, 'Robert Jones': {'wikidata': 'http://www.wikidata.org/entity/Q18630239', 'dates': '1570-1617'}, 'Jhan Gero': {'wikidata': 'http://www.wikidata.org/entity/Q1657883', 'dates': '1518-1555'}, 'Alfonso Fontanelli': {'wikidata': 'http://www.wikidata.org/entity/Q2062267', 'dates': '1557-1622'}, 'Giovanni Ghizzolo': {'wikidata': 'http://www.wikidata.org/entity/Q3107205', 'dates': '1580-1625'}, 'Giovanni Ferretti': {'wikidata': 'http://www.wikidata.org/entity/Q3107173', 'dates': '1540-1609'}, 'Ellis Gibbons': {'wikidata': 'http://www.wikidata.org/entity/Q3723541', 'dates': '1573-1603'}, 'Giacomo Fogliano': {'wikidata': 'http://www.wikidata.org/entity/Q3805961', 'dates': '1468-1548'}, 'John Holmes': {'wikidata': 'http://www.wikidata.org/entity/Q3809346', 'dates': '1629'}, 'Thomas Greaves': {'wikidata': 'http://www.wikidata.org/entity/Q3990629', 'dates': '1604'}, 'Giulio Fiesco': {'wikidata': 'http://www.wikidata.org/entity/Q5565265', 'dates': '1520-1586'}, 'Edward Johnson': {'wikidata': 'http://www.wikidata.org/entity/Q5343803', 'dates': '1545-1601'}, 'John Hilton the elder': {'wikidata': 'http://www.wikidata.org/entity/Q6239329', 'dates': '1569-1608'}, 'Maistre Jhan': {'wikidata': 'http://www.wikidata.org/entity/Q6737089', 'dates': '1485-1538'}, 'Scipione Lacorcia': {'wikidata': 'http://www.wikidata.org/entity/Q7434030', 'dates': '1585-1620'}, 'Sebastiano Festa': {'wikidata': 'http://www.wikidata.org/entity/Q7442579', 'dates': '1495-1524'}, 'Paolo Isnardi': {'wikidata': 'http://www.wikidata.org/entity/Q16863445', 'dates': '1536-1596'}, 'Antonio Gualtieri': {'wikidata': 'http://www.wikidata.org/entity/Q16054417', 'dates': '1649'}, 'Johann Lyttich': {'wikidata': 'http://www.wikidata.org/entity/Q20003638', 'dates': '1581-1611'}, 'Gabriele Fallamero': {'wikidata': 'http://www.wikidata.org/entity/Q112456781', 'dates': '1584'}, 'Andrea Feliciani': {'wikidata': 'http://www.wikidata.org/entity/Q55227467', 'dates': '1596'}, 'Camillo Lambardi': {'wikidata': 'http://www.wikidata.org/entity/Q50316330', 'dates': '1560-1634'}, 'Thomas Hunt': {'wikidata': 'http://www.wikidata.org/entity/Q99237806', 'dates': '1580-1658'}, 'Giovan Battista Leonetti': {'wikidata': 'http://www.wikidata.org/entity/Q107995775', 'dates': '1575-1617'}, 'Girolamo Ferrari': {'wikidata': 'http://www.wikidata.org/entity/Q113412847', 'dates': '1600-1664'}, 'Henry Purcell': {'wikidata': 'http://www.wikidata.org/entity/Q9695', 'dates': '1659-1695'}, 'Claudio Monteverdi': {'wikidata': 'http://www.wikidata.org/entity/Q53068', 'dates': '1567-1643'}, 'Girolamo Parabosco': {'wikidata': 'http://www.wikidata.org/entity/Q29300', 'dates': '1524-1557'}, 'Giovanni Antonio Rigatti': {'wikidata': 'http://www.wikidata.org/entity/Q142515', 'dates': '1615-1649'}, 'Cipriano de Rore': {'wikidata': 'http://www.wikidata.org/entity/Q316074', 'dates': '1515-1565'}, 'Francesco Rasi': {'wikidata': 'http://www.wikidata.org/entity/Q363011', 'dates': '1574-1620'}, 'Claudio Merulo': {'wikidata': 'http://www.wikidata.org/entity/Q380517', 'dates': '1533-1604'}, 'Jacob Regnart': {'wikidata': 'http://www.wikidata.org/entity/Q267177', 'dates': '1540-1599'}, 'Michael Praetorius': {'wikidata': 'http://www.wikidata.org/entity/Q108278', 'dates': '1571-1621'}, 'Philippe de Monte': {'wikidata': 'http://www.wikidata.org/entity/Q277029', 'dates': '1521-1603'}, 'Thomas Morley': {'wikidata': 'http://www.wikidata.org/entity/Q438141', 'dates': '1557-1603'}, 'Giovanni Maria Nanino': {'wikidata': 'http://www.wikidata.org/entity/Q706287', 'dates': '1543-1607'}, 'Thomas Ravenscroft': {'wikidata': 'http://www.wikidata.org/entity/Q634595', 'dates': '1590-1633'}, 'Salamone Rossi': {'wikidata': 'http://www.wikidata.org/entity/Q713436', 'dates': '1570-1630'}, 'Giovanni Bernardino Nanino': {'wikidata': 'http://www.wikidata.org/entity/Q723537', 'dates': '1560-1623'}, 'Asprilio Pacelli': {'wikidata': 'http://www.wikidata.org/entity/Q738001', 'dates': '1570-1623'}, 'Mogens Pedersøn': {'wikidata': 'http://www.wikidata.org/entity/Q962427', 'dates': '1585-1623'}, 'Giovanni Battista Mosto': {'wikidata': 'http://www.wikidata.org/entity/Q1119331', 'dates': '1549-1596'}, 'Marco Scacchi': {'wikidata': 'http://www.wikidata.org/entity/Q1357654', 'dates': '1600-1687'}, 'Giovanni Priuli': {'wikidata': 'http://www.wikidata.org/entity/Q1445743', 'dates': '1575-1629'}, 'Vincenzo Ruffo': {'wikidata': 'http://www.wikidata.org/entity/Q1494084', 'dates': '1508-1587'}, 'Giovanni Rovetta': {'wikidata': 'http://www.wikidata.org/entity/Q1526347', 'dates': '1596-1668'}, 'Melchior Neusidler': {'wikidata': 'http://www.wikidata.org/entity/Q1919164', 'dates': '1531-1590'}, 'Peter Philips': {'wikidata': 'http://www.wikidata.org/entity/Q1930460', 'dates': '1561-1628'}, 'Mario Savioni': {'wikidata': 'http://www.wikidata.org/entity/Q1899013', 'dates': '1608-1685'}, 'Martino Pesenti': {'wikidata': 'http://www.wikidata.org/entity/Q1905637', 'dates': '1600-1648'}, 'Andreas Pevernage': {'wikidata': 'http://www.wikidata.org/entity/Q2159084', 'dates': '1543-1591'}, 'Francis Pilkington': {'wikidata': 'http://www.wikidata.org/entity/Q2462847', 'dates': '1562-1638'}, 'Angelo Notari': {'wikidata': 'http://www.wikidata.org/entity/Q2849490', 'dates': '1566-1663'}, 'Giovanni Domenico da Nola': {'wikidata': 'http://www.wikidata.org/entity/Q2395209', 'dates': '1510-1592'}, 'Erasmo Marotta': {'wikidata': 'http://www.wikidata.org/entity/Q3731086', 'dates': '1578-1641'}, 'Jan Nasco': {'wikidata': 'http://www.wikidata.org/entity/Q3767806', 'dates': '1510-1561'}, 'Pietro Maria Marsolo': {'wikidata': 'http://www.wikidata.org/entity/Q3904046', 'dates': '1580-1615'}, 'Pomponio Nenna': {'wikidata': 'http://www.wikidata.org/entity/Q3907903', 'dates': '1550- 1613'}, 'Robert Ramsey': {'wikidata': 'http://www.wikidata.org/entity/Q3938401', 'dates': '1590-1644'}, 'Alessandro Orologio': {'wikidata': 'http://www.wikidata.org/entity/Q3610181', 'dates': '1550-1633'}, 'Andrea Rota': {'wikidata': 'http://www.wikidata.org/entity/Q3616007', 'dates': '1553-1597'}, 'John Milton': {'wikidata': 'http://www.wikidata.org/entity/Q110935304', 'dates': '1608-1674'}, 'John Mundy': {'wikidata': 'http://www.wikidata.org/entity/Q3809503', 'dates': '1555-1630'}, 'David Sacerdote': {'wikidata': 'http://www.wikidata.org/entity/Q5239412', 'dates': '1550-1625'}, 'Benedetto Pallavicino': {'wikidata': 'http://www.wikidata.org/entity/Q4887229', 'dates': '1551-1601'}, 'Claudio Pari': {'wikidata': 'http://www.wikidata.org/entity/Q5129392', 'dates': '1574-1619'}, 'Galeazzo Sabbatini': {'wikidata': 'http://www.wikidata.org/entity/Q5518314', 'dates': '1597-1662'}, 'Giovan Leonardo Primavera': {'wikidata': 'http://www.wikidata.org/entity/Q5563608', 'dates': '1540-1585'}, 'Ginés de Morata': {'wikidata': 'http://www.wikidata.org/entity/Q5880038', 'dates': '1576'}, 'Rinaldo del Mel': {'wikidata': 'http://www.wikidata.org/entity/Q7334576', 'dates': '1554-1598'}, 'Stefano Rossetto': {'wikidata': 'http://www.wikidata.org/entity/Q7606486', 'dates': '1560-1583'}, 'Stephan Otto': {'wikidata': 'http://www.wikidata.org/entity/Q7608155', 'dates': '1603-1656'}, 'Tiburtio Massaino': {'wikidata': 'http://www.wikidata.org/entity/Q7800551', 'dates': '1550-1608'}, 'Francesco Portinaro': {'wikidata': 'http://www.wikidata.org/entity/Q16863487', 'dates': '1520-1578'}, 'Hubert Naich': {'wikidata': 'http://www.wikidata.org/entity/Q16863472', 'dates': '1513-1546'}, 'Walter Porter': {'wikidata': 'http://www.wikidata.org/entity/Q18759427', 'dates': '1587-1659'}, 'Jeremy Savile': {'wikidata': 'http://www.wikidata.org/entity/Q18810825', 'dates': '1651-1665'}, 'Ippolito Sabino': {'wikidata': 'http://www.wikidata.org/entity/Q49361116', 'dates': '1550-1593'}, 'Ambrosius Metzger': {'wikidata': 'http://www.wikidata.org/entity/Q55847929', 'dates': '1573-1632'}, 'Daniel Norcome': {'wikidata': 'http://www.wikidata.org/entity/Q55903969', 'dates': '1601'}, 'Pietro Pace': {'wikidata': 'http://www.wikidata.org/entity/Q63210400', 'dates': '1559-1622'}, 'Giovan Domenico Montella': {'wikidata': 'http://www.wikidata.org/entity/Q55998400', 'dates': '1570-1607'}, 'Grammatio Metallo': {'wikidata': 'http://www.wikidata.org/entity/Q61061769', 'dates': '1540-1615'}, 'Filippo Nicoletti': {'wikidata': 'http://www.wikidata.org/entity/Q87254387', 'dates': '1577-1620'}, 'Domenico Micheli': {'wikidata': 'http://www.wikidata.org/entity/Q87642969', 'dates': '1540-1590'}, 'Bartolomeo Ratti': {'wikidata': 'http://www.wikidata.org/entity/Q88300377', 'dates': '1565-1634'}, 'Biagio Pesciolini': {'wikidata': 'http://www.wikidata.org/entity/Q101439336', 'dates': '1610'}, 'Santi Orlandi': {'wikidata': 'http://www.wikidata.org/entity/Q102035932', 'dates': '1619'}, 'Nathaniel Patrick': {'wikidata': 'http://www.wikidata.org/entity/Q98915412', 'dates': '1590'}, 'Johann Hermann Schein': {'wikidata': 'http://www.wikidata.org/entity/Q57827', 'dates': '1586-1630'}, 'Ludwig Senfl': {'wikidata': 'http://www.wikidata.org/entity/Q110133949', 'dates': '1490-1543'}, 'Adrian Willaert': {'wikidata': 'http://www.wikidata.org/entity/Q312615', 'dates': '1490-1562'}, 'Barbara Strozzi': {'wikidata': 'http://www.wikidata.org/entity/Q232235', 'dates': '1619-1677'}, 'Alessandro Stradella': {'wikidata': 'http://www.wikidata.org/entity/Q314765', 'dates': '1639-1682'}, 'Heinrich Schütz': {'wikidata': 'http://www.wikidata.org/entity/Q153643', 'dates': '1585-1672'}, 'Orazio Vecchi': {'wikidata': 'http://www.wikidata.org/entity/Q84146', 'dates': '1550-1605'}, 'Gioseffo Zarlino': {'wikidata': 'http://www.wikidata.org/entity/Q365805', 'dates': '1517-1590'}, 'Vincenzo Ugolini': {'wikidata': 'http://www.wikidata.org/entity/Q330488', 'dates': '1570-1638'}, 'Jan Pieterszoon Sweelinck': {'wikidata': 'http://www.wikidata.org/entity/Q110294', 'dates': '1562-1621'}, 'Alessandro Scarlatti': {'wikidata': 'http://www.wikidata.org/entity/Q183087', 'dates': '1660-1725'}, 'Jan Tollius': {'wikidata': 'http://www.wikidata.org/entity/Q284127', 'dates': '1550-1603'}, 'John Ward': {'wikidata': 'http://www.wikidata.org/entity/Q447968', 'dates': '1571-1638'}, 'Giaches de Wert': {'wikidata': 'http://www.wikidata.org/entity/Q591939', 'dates': '1535-1596'}, 'Bartolomeo Tromboncino': {'wikidata': 'http://www.wikidata.org/entity/Q809518', 'dates': '1470-1534'}, 'Philippe Verdelot': {'wikidata': 'http://www.wikidata.org/entity/Q714219', 'dates': '1480-1530'}, 'Johann Stadlmayr': {'wikidata': 'http://www.wikidata.org/entity/Q472907', 'dates': '1575-1648'}, 'Thomas Weelkes': {'wikidata': 'http://www.wikidata.org/entity/Q553908', 'dates': '1575-1623'}, 'John Wilbye': {'wikidata': 'http://www.wikidata.org/entity/Q682089', 'dates': '1574-1638'}, 'Nicola Vicentino': {'wikidata': 'http://www.wikidata.org/entity/Q944773', 'dates': '1511-1576'}, 'Cornelis Schuyt': {'wikidata': 'http://www.wikidata.org/entity/Q749364', 'dates': '1557-1616'}, 'Francesco Turini': {'wikidata': 'http://www.wikidata.org/entity/Q1441256', 'dates': '1589-1656'}, 'Thomas Tomkins': {'wikidata': 'http://www.wikidata.org/entity/Q968698', 'dates': '1572-1656'}, 'Francesco Soriano': {'wikidata': 'http://www.wikidata.org/entity/Q1376058', 'dates': '1548-1621'}, 'Alessandro Striggio': {'wikidata': 'http://www.wikidata.org/entity/Q1377115', 'dates': '1536-1592'}, 'Johannes Schultz': {'wikidata': 'http://www.wikidata.org/entity/Q1406864', 'dates': '1582-1653'}, 'Thomas Whythorne': {'wikidata': 'http://www.wikidata.org/entity/Q2124600', 'dates': '1528-1596'}, 'Hubert Waelrant': {'wikidata': 'http://www.wikidata.org/entity/Q2664749', 'dates': '1516-1595'}, 'Annibale Zoilo': {'wikidata': 'http://www.wikidata.org/entity/Q2920312', 'dates': '1537-1592'}, 'Filippo Vitali': {'wikidata': 'http://www.wikidata.org/entity/Q3071924', 'dates': '1590-1653'}, 'Ascanio Trombetti': {'wikidata': 'http://www.wikidata.org/entity/Q2866349', 'dates': '1544-1590'}, 'Pietro Vinci': {'wikidata': 'http://www.wikidata.org/entity/Q3388207', 'dates': '1525-1584'}, 'Edmund Turges': {'wikidata': 'http://www.wikidata.org/entity/Q3719486', 'dates': '1450-1507'}, 'Paolo Virchi': {'wikidata': 'http://www.wikidata.org/entity/Q3894585', 'dates': '1551-1610'}, 'Claudio Veggio': {'wikidata': 'http://www.wikidata.org/entity/Q5129425', 'dates': '1510-1543'}, 'Girolamo Scotto': {'wikidata': 'http://www.wikidata.org/entity/Q5564788', 'dates': '1505-1572'}, 'Pietro Taglia': {'wikidata': 'http://www.wikidata.org/entity/Q7193041', 'dates': '1555-65'}, 'Stefano Venturi del Nibbio': {'wikidata': 'http://www.wikidata.org/entity/Q7606503', 'dates': '1592-1600'}, 'Antonio Francesco Tenaglia': {'wikidata': 'http://www.wikidata.org/entity/Q10954739', 'dates': '1612-1672'}, 'Orfeo Vecchi': {'wikidata': 'http://www.wikidata.org/entity/Q11939478', 'dates': '1551-1603'}, 'Thomas Vautor': {'wikidata': 'http://www.wikidata.org/entity/Q11951781', 'dates': '1600-1620'}, 'Bartolomeo Spontone': {'wikidata': 'http://www.wikidata.org/entity/Q21001517', 'dates': '1530-1592'}, 'Melchior Schramm': {'wikidata': 'http://www.wikidata.org/entity/Q24049612', 'dates': '1553-1619'}, 'Agostino Soderini': {'wikidata': 'http://www.wikidata.org/entity/Q77535913', 'dates': '1575-1607'}, 'Camillo Zanotti': {'wikidata': 'http://www.wikidata.org/entity/Q84562722', 'dates': '1545-1591'}, 'Cesare Zoilo': {'wikidata': 'http://www.wikidata.org/entity/Q109265169', 'dates': '1584-1622'}, 'Flaminio Tresti': {'wikidata': 'http://www.wikidata.org/entity/Q110222853', 'dates': '1560-1613'}}

This allowed to add only one composer to our collection. Indeed, Wikidata has very few instances of madrigals registered.

We tried to query the endpoint to find more information, for example setting madrigals as the movement (P135) or the genre (P136) to which the composer belonged, but none was conclusive.

So we remain with our dataset of 245 individuals.

### Adding metadata for our composers
Now let's add some information: Nationality, gender, birthplace, place of death, languages spoken and instruments played.

In [43]:
labels = {"genderLabel":"gender", "citizenshipLabel":"citizenship", "birthplaceLabel":"birthplace", "birthcountryLabel":"birth country", "deathplaceLabel":"place of death", "deathcountryLabel":"country of death", "languageLabel":"language", "instrumentLabel":"instrument"}

In [44]:
for composer in dict_composers_wd:
    my_SPARQL_query = """
    SELECT ?genderLabel ?citizenshipLabel ?birthplaceLabel ?birthcountryLabel ?deathplaceLabel ?deathcountryLabel ?languageLabel ?instrumentLabel WHERE {
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P21 ?birth_date } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P21 ?gender } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P27 ?citizenship } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P19 ?birthplace .
                       ?birthplace wdt:P17 ?birthcountry} .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P20 ?deathplace .
                       ?deathplace wdt:P17 ?deathcountry} .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P1412 ?language } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P1303 ?instrument } .
              SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }  # labels in English
    }

    """
    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(my_SPARQL_query)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    # manipulate the result
    for result in results["results"]["bindings"]:
        for label in labels:
            if label in result:
                if labels[label] in ["citizenship", "language", "instrument"]:
                    if labels[label] in dict_composers_wd[composer] and result[label]["value"] not in dict_composers_wd[composer][labels[label]]:
                        if type(dict_composers_wd[composer][labels[label]]) != list:
                            dict_composers_wd[composer][labels[label]] = list([dict_composers_wd[composer][labels[label]]])
                        dict_composers_wd[composer][labels[label]].append(result[label]["value"])
                    elif labels[label] not in dict_composers_wd[composer]:
                        dict_composers_wd[composer][labels[label]] = result[label]["value"]
                else:
                    dict_composers_wd[composer][labels[label]] = result[label]["value"]

In [45]:
df_composers = pd.DataFrame.from_dict(dict_composers_wd, orient="index")


In [46]:
df_composers.reset_index(inplace=True)
df_composers = df_composers.rename(columns = {'index':'name'})
df_composers

,name,wikidata,dates,gender,citizenship,birthplace,birth country,place of death,country of death,language,instrument
0,Nicolaus Bruhns,http://www.wikidata.org/entity/Q57369,1665-1697,male,"[Denmark, Duchy of Schleswig]",Schwabstedt,Germany,Husum,Germany,German,organ
1,Marc-Antoine Charpentier,http://www.wikidata.org/entity/Q55524,1643-1704,male,Kingdom of France,Paris,France,Paris,France,French,"[organ, voice]"
2,Girolamo Dalla Casa,http://www.wikidata.org/entity/Q354023,1543-1601,male,Republic of Venice,Udine,Italy,Venice,Italy,NaN,NaN
3,Arcangelo Corelli,http://www.wikidata.org/entity/Q164475,1653-1713,male,Papal States,Fusignano,Italy,Rome,Italy,"[Latin, Italian]",violin
4,John Bennet,http://www.wikidata.org/entity/Q374718,1575-1614,male,England,Lancashire,United Kingdom,NaN,NaN,English,NaN
...,...,...,...,...,...,...,...,...,...,...,...
240,Agostino Soderini,http://www.wikidata.org/entity/Q77535913,1575-1607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,Camillo Zanotti,http://www.wikidata.org/entity/Q84562722,1545-1591,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,Cesare Zoilo,http://www.wikidata.org/entity/Q109265169,1584-1622,male,Papal States,NaN,NaN,Rome,Italy,NaN,NaN
243,Flaminio Tresti,http://www.wikidata.org/entity/Q110222853,1560-1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We save the final dataset in json and csv in order to use it for visualizations later on.

In [47]:
df_composers.to_json("composers.json")
df_composers.to_csv("composers.csv")

# Working on works and cleaning the British Library dataset
We transform the dictionary into a dataframe and merge it to the one with all instances of works, selecting only the columns of interest for us. 

In [76]:
df_comp_madri = pd.DataFrame.from_dict(comp_madrigals, orient="index")
df_comp_madri = df_comp_madri.reset_index()
df_comp_madri.columns=["Composer", "Composer life dates"]

In [77]:
df_madri = df_madri.merge(df_comp_madri, how="left", on="Composer")
df_madri

,BL record ID,Composer,Title,Standardised title,Other titles,Other names,Publication date (standardised),Publication date (not standardised),Country of publication,Place of publication,...,Referenced in,Subject/genre terms,Physical description,Series title,Number within series,ISBN,ISMN,Publisher number,BL shelfmark,Composer life dates
0,4166081,"Adlam, Frank",Winter stern hath loosed his Grip. <Madrigal f...,NaN,NaN,NaN,1913,1913,England,London,...,NaN,NaN,"3 pages, 8°",Choruses for equal Voices,no. 1387 [Choruses for equal Voices],NaN,NaN,NaN,mDON1781 ; E.861./1387,NaN
1,4166165,"Adler, Samuel",Three Madrigals. For four-part chorus of mixed...,NaN,NaN,NaN,1958,1958,United States,New York,...,NaN,NaN,"11 pages, 8°",NaN,NaN,NaN,NaN,NaN,F.1744.v.(1.),NaN
2,4166293,"Adriani, Francesco",Il Primo Libro de Madrigali a Cinque Voci ... ...,NaN,NaN,NaN,1570,1570,Italy,Vinegia,...,NaN,NaN,4°,NaN,NaN,NaN,NaN,NaN,D.148,1539-1575
3,4166294,"Adriani, Francesco",Il Secondo Libro de Madrigali a Cinque Voci .....,NaN,NaN,NaN,1570,1570,Italy,Vinegia,...,NaN,NaN,4°,NaN,NaN,NaN,NaN,NaN,D.148.a,1539-1575
4,4166295,"Adriansen, Emanuel","Luitmuziek ... Een keuze van fantasieën, danse...",NaN,NaN,"Spiessons, Godelieve",1966,1966,Belgium,Antwerpen,...,NaN,NaN,"xvi, 97 pages, facsimiles, folio",Monumenta musicæ belgicæ,no. 10 [Monumenta musicæ belgicæ],NaN,NaN,NaN,H.15,1554-1604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,16837423,"Bennet, John",All creatures now are merry-minded : (SSATB) ;...,NaN,Triumphes of Oriana,"Wilkes, Roger, (Musician) [editor]",2014,©2014,England,Cheshire,...,NaN,"Choruses, Secular (Mixed voices)--Scores ; Mad...","1 score (12 pages) + 9 parts, 30 cm",Editions of renaissance music,NaN,NaN,9790570570317 ; 9790570570324,TR004,G.1535.l.(1.),1575-1614
3412,16837433,"Jones, Robert","Fair Oriana, seeming to wink : (SSAATB) : from...",NaN,Triumphes of Oriana,"Wilkes, Roger, (Musician) [editor]",2014,©2014,England,Cheshire,...,NaN,"Choruses, Secular (Mixed voices)--Scores ; Mad...","1 score (12 pages) + 14 parts, 30 cm",Editions of renaissance music,NaN,NaN,9790570570300,TR021,G.1535.l.(4.),1570-1617
3413,16840548,"Jackson, William","Ten duets : for sopranos, tenors, or soprano &...",Vocal music. Selections,Duets ; Canzonets ; Pastorals ; Madrigals,"Jackson, William, op. 9 ; Jackson, William, op...",2014,©2013,England,Richmond,...,NaN,Vocal duets with continuo,"2 scores (44, 39 pages), 30 cm",NaN,NaN,NaN,9790708105701,Jac 1,G.804.ll.(11.),NaN
3414,16874797,"Tomkins, Thomas",Dear Lord of life,NaN,NaN,"Burke, James [editor]",2014,©2014,England,Oxford,...,NaN,"Anthems ; Choruses, Sacred (Mixed voices, 6 pa...","1 score (15 pages), 30 cm",Church Music Society reprints,no. 132 [Church Music Society reprints],9780193954014,NaN,NaN,E.1617./132,1572-1656


In [78]:
df_madri=df_madri.dropna(subset=['Composer life dates'])

In [79]:
df_madri = df_madri[["BL record ID", "Title", "Standardised title", "Other titles", "Composer", "Composer life dates", "Other names", "Publication date (standardised)", "Country of publication", "Place of publication", "Publisher", "Notes", "Contents","Subject/genre terms"]]

In [17]:
df_madri.to_csv("madrigals_w_dates_comp_records.csv")

Filtering on publication date to try to clean data and extract only relevant madrigals

In [80]:
df_madri=df_madri.dropna(subset=['Publication date (standardised)'])
df_madri['Publication date (standardised)'] = df_madri['Publication date (standardised)'].astype("int")
df_madri = df_madri.sort_values(by=['Publication date (standardised)'])

By examining our dataset, we noticed that there were a few more entries for madrigals until the end of the 1600s (until 1678) before a jump to 1762 where the genre of madrigals changes. So we decided to include these in the final dataset

In [1]:
df_madri = df_madri.loc[df_madri['Publication date (standardised)'] <= 1678]
df_madri

NameError: name 'df_madri' is not defined

In [20]:
df_madri.to_csv("madri_composed_bef_1678_records.csv")

Form Wikidata: Add entity for the publication place.
Add entity for the composer, if exists -> add nationality. other?
Add entity for the publisher if exists

Add a column with number of voices

I stopped at line 424 in the excel

In [24]:
dictionary_voices = {"uno":1, "due":2, "doi":2, "dui":2, "tre":3, "quattro":4, "quatro":4, "cinque":5, "sei":6, "sette":7,"otto":8,"nove":9,"dieci":10,"undici":11,"dodici":12,
                    "quatuor":4, "quinque":5, "sex":6, "octo":8, "decem":10, "cinqve":5, "sola":1, "solo":1, "iiii":4,
                    "one":1, "two":2, "three":3, "four":4, "foure":4, "five":5, "six":6, "seven":7, "eight":8,
                    "un":1,"deux":2, "trois":3, "quatre":4, "cinq":5, "sept":7, "huit":8,
                    "eins":1, "zwei":2, "drei":3, "vier":4, "fünf":5, "sechs":6, "sieben":7, "acht":8, "neun":9, "zehn":10}

In [142]:
import numpy as np
output = df_madri.copy(deep=True)
output['Voices']=''
for row in output.itertuples():
    title_splitted = row[2].lower().split()
    for el in title_splitted:
        el = el.strip(" .,():")
        if el in dictionary_voices:
            if type(output.loc[row[0], 'Voices']) is not list:
                output.at[row[0], 'Voices']=list([dictionary_voices[el]])
            else:
                output.loc[row[0], 'Voices'].append(int(dictionary_voices[el]))

      BL record ID                                              Title  \
2114       4715714  Del primo Libro de Madrigali di Verdelotto. A[...   
2118       4715718  Intauolatura de li Madrigali di Verdelotto da ...   
62         4175561                       Il quarto libro di madrigali   
61         4175560                       Il terzo libro dei madrigali   
59         4175558  Il Secondo Libro di Madrigali d'Arcadelt nouam...   

                      Standardised title Other titles            Composer  \
2114                                 NaN          NaN  Verdelot, Philippe   
2118  Madrigali a quatro voci. Lib. 1, 2          NaN  Verdelot, Philippe   
62         Madrigals. voices (4). book 4          NaN     Arcadelt, Jacob   
61         Madrigals. voices (4). book 3          NaN     Arcadelt, Jacob   
59                                   NaN          NaN     Arcadelt, Jacob   

     Composer life dates                                        Other names  \
2114           1480

In [143]:
output.to_csv("df_madri_w_voices.csv")